In [29]:
from keras.layers import Input,Conv2D,GlobalAveragePooling2D,concatenate,Dense
from keras.models import Model
import keras.backend as K
import numpy as np
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.losses import mean_absolute_error
import tensorflow as tf

m_e = .2
m_d = .3

def Le(true,pred):    
    return m_e + K.sum(K.square(pred[0] - pred[1]),axis=-1) - K.sum(K.square(pred[0] - pred[2]),axis=-1)
    
def Ld(true,pred):
    return 0

def Triplet_loss(true,pred):
    return 0

def myloss(y_true,y_pred): 
    print tf.shape(y_true)
    print tf.shape(y_pred)
    return K.square(K.sum(y_pred[0]-y_pred[1],axis=-1)) 

def create_shared_weights(conv1, conv2,conv3, input_shape):
    with K.name_scope(conv1.name):
        conv1.build(input_shape)
    with K.name_scope(conv2.name):
        conv2.build(input_shape)
    with K.name_scope(conv3.name):
        conv3.build(input_shape)
    conv2.kernel = conv1.kernel
    conv2.bias = conv1.bias
    conv2._trainable_weights = []
    conv2._trainable_weights.append(conv2.kernel)
    conv2._trainable_weights.append(conv2.bias)
    conv3.kernel = conv1.kernel                               
    conv3.bias = conv1.bias                      
    conv3._trainable_weights = []                
    conv3._trainable_weights.append(conv3.kernel)
    conv3._trainable_weights.append(conv3.bias)
 # check if weights are successfully shared
input_left = Input(shape=(299, 299, 3))
input_center = Input(shape=(299, 299, 3))
input_right = Input(shape=(299, 299, 3))


leftname = input_left.name
centername = input_center.name
rightname = input_right.name


conv1 = Conv2D(64, 3, padding='same')
conv2 = Conv2D(64, 3, padding='valid')
conv3 = Conv2D(64, 3, padding='valid')

create_shared_weights(conv1, conv2, conv3, input_center._keras_shape)
print(conv2.weights == conv1.weights)#True
print(conv3.weights == conv1.weights)
# check if weights are equal after model fitting
left = conv1(input_left)
right = conv2(input_right)
center = conv3(input_center)

left = GlobalAveragePooling2D()(left)
right = GlobalAveragePooling2D()(right)
center = GlobalAveragePooling2D()(center)

oleftname = left.name
ocentername = center.name
orightname = right.name

#merged = concatenate([left, right])
#output = Dense(1)(merged)
model = Model([input_left,input_right,input_center], [left,right,center])
#model.compile(loss='binary_crossentropy', optimizer='adam')

IMAGES_NUMBER = 50

X = {
    leftname[:-2]:10*np.ones((IMAGES_NUMBER, 299, 299, 3)),
    centername[:-2]:50*np.ones((IMAGES_NUMBER, 299, 299, 3)),
    rightname[:-2]:100*np.ones((IMAGES_NUMBER, 299, 299, 3))
    }

Y = {
    oleftname[:-2]:np.random.random((IMAGES_NUMBER,64)),
    ocentername[:-2]:np.random.random((IMAGES_NUMBER,64)),
    orightname[:-2]:np.random.random((IMAGES_NUMBER,64))
    }

#Y = np.random.randint(2, size=5)
#model.fit(X, Y)
plot_model(model=model,show_layer_names=True,show_shapes=True,to_file='modelWeigthsshared.png')
print([np.all(w1 == w2) for w1, w2 in zip(conv1.get_weights(), conv2.get_weights())])
model.compile(loss=myloss,
              metrics=['mae'],
              optimizer=Adam())          
history = model.fit(batch_size=4,x=X,y=[np.ones((IMAGES_NUMBER,64)),2*np.ones((IMAGES_NUMBER,64)),3*np.ones((IMAGES_NUMBER,64))],epochs=5)


True
True
[True, True]
Tensor("loss_8/global_average_pooling2d_28_loss/Shape:0", shape=(2,), dtype=int32)
Tensor("loss_8/global_average_pooling2d_28_loss/Shape_1:0", shape=(2,), dtype=int32)
Tensor("loss_8/global_average_pooling2d_29_loss/Shape:0", shape=(2,), dtype=int32)
Tensor("loss_8/global_average_pooling2d_29_loss/Shape_1:0", shape=(2,), dtype=int32)
Tensor("loss_8/global_average_pooling2d_30_loss/Shape:0", shape=(2,), dtype=int32)
Tensor("loss_8/global_average_pooling2d_30_loss/Shape_1:0", shape=(2,), dtype=int32)
Epoch 1/5
50/50 [==============================] - 1s 19ms/step - loss: 0.0000e+00 - global_average_pooling2d_28_loss: 0.0000e+00 - global_average_pooling2d_29_loss: 0.0000e+00 - global_average_pooling2d_30_loss: 0.0000e+00 - global_average_pooling2d_28_mean_absolute_error: 3.0218 - global_average_pooling2d_29_mean_absolute_error: 29.0934 - global_average_pooling2d_30_mean_absolute_error: 14.7840
Epoch 2/5
50/50 [==============================] - 1s 10ms/step - loss: 0

In [7]:
out1 = model.predict(x=X)
np.shape(out1)

(3, 50, 64)

In [12]:
out1[0]

array([[-2.9332733 ,  4.2688212 ,  4.890018  , ...,  4.093476  ,
         2.5398412 ,  0.20210178],
       [-2.9332733 ,  4.2688212 ,  4.890018  , ...,  4.093476  ,
         2.5398412 ,  0.20210178],
       [-2.9332733 ,  4.2688212 ,  4.890018  , ...,  4.093476  ,
         2.5398412 ,  0.20210178],
       ...,
       [-2.9332733 ,  4.2688212 ,  4.890018  , ...,  4.093476  ,
         2.5398412 ,  0.20210178],
       [-2.9332733 ,  4.2688212 ,  4.890018  , ...,  4.093476  ,
         2.5398412 ,  0.20210178],
       [-2.9332733 ,  4.2688212 ,  4.890018  , ...,  4.093476  ,
         2.5398412 ,  0.20210178]], dtype=float32)

In [19]:
yy = [np.ones((IMAGES_NUMBER,64)),2*np.ones((IMAGES_NUMBER,64)),3*np.ones((IMAGES_NUMBER,64))]
out = model.evaluate(x=X,y=yy,verbose=1,batch_size=4)

50/50 [==============================] - 1s 10ms/step


In [14]:
out

[0.0, 0.0, 0.0, 0.0, 2.1512229442596436, 21.041837692260742, 10.46918773651123]

In [32]:
K.gradients




<function tensorflow.python.ops.gradients_impl.gradients>

In [30]:
# sess = tf.Session()
# loss_out = sess.run(Le(out))
# sess.close()
